In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import requests
import pickle
import tensorflow as tf
import random 
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

url='https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
text=requests.get(url).text.lower()
print("Total characters: ",len(text))

2025-09-25 15:37:38.352904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758814658.546403      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758814658.600052      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Total characters:  1115394


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text.split())  
total_words = len(tokenizer.word_index) + 1
print("Total unique words:", total_words)

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


index_to_word = {index: word for word, index in tokenizer.word_index.items()}
with open("index_to_word.pkl", "wb") as f:
    pickle.dump(index_to_word, f)

Total unique words: 12633


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_seq=token_list[:i+1]
        input_sequences.append(n_gram_seq)

In [5]:
max_seq_len=max(len(seq) for seq in input_sequences)
input_sequences=pad_sequences(input_sequences,maxlen=max_seq_len,padding='pre')

print(input_sequences)

X,y=input_sequences[:,:-1],input_sequences[:,-1]
# X → all words except the last one (input).
# y → last word (label/output).
# to_categorical → one-hot encode labels for classification.
import tensorflow as tf
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

print("Training smaples :",X.shape)

[[   0    0    0 ...    0   88  269]
 [   0    0    0 ...    0  139   35]
 [   0    0    0 ...  139   35  969]
 ...
 [   0    0    0 ...    0 1062   25]
 [   0    0    0 ... 1062   25  131]
 [   0    0    0 ...   25  131 1828]]
Training smaples : (171312, 15)


In [6]:
from keras.models import *
from keras.layers import *
model = Sequential([
    Embedding(total_words, 200, input_length=max_seq_len-1),   
    LSTM(256, return_sequences=True),
    Dropout(0.3),
    LSTM(128),
    Dropout(0.3),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(total_words, activation="softmax")
])
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
history = model.fit(X, y, batch_size=64, epochs=100, verbose=1, callbacks=[early_stop])

loss,acc=model.evaluate(X,y)
print("Acc: ",acc)

model.save("Next_word.h5")
print("Model and tokenizer saved successfully!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1758814673.434260      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758814673.434928      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


I0000 00:00:1758814708.507878      62 cuda_dnn.cc:529] Loaded cuDNN version 90300


2677/2677 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.0319 - loss: 7.0213
Epoch 2/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0487 - loss: 6.4684
Epoch 3/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0654 - loss: 6.2126
Epoch 4/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0771 - loss: 6.0363
Epoch 5/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0838 - loss: 5.9040
Epoch 6/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0893 - loss: 5.7935
Epoch 7/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0931 - loss: 5.7017
Epoch 8/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0960 - loss: 5.6109
Epoch 9/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.0999 - loss: 5.5417
Epoch 10/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.1014 - loss: 5.4728
Epoch 11/100
2677/2677 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.1043 - loss: 5.4169
Epoch 12/

In [7]:
def predict_next_word(seed_text, next_words=100):
    """Generate next words for a given seed text."""
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences(seed_text.split())
        token_list = [t[0] for t in token_list if t]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted_index = np.argmax(model.predict(token_list, verbose=0), axis=1)[0]
        output_word = index_to_word.get(predicted_index, "")
        seed_text += " " + output_word
    return seed_text



In [8]:
test_seed = "We are accounted"
generated_text = predict_next_word(test_seed, next_words=100)
print("Generated Text:\n", generated_text)

Generated Text:
 We are accounted one of you to london and i am not so much used to make the feast of her old babes of changing day and dried at the world whereon the duke of hereford i have you not to be a man of the next man i am wont to die and the duke of the yoke of changing time to be a man and the duke of somerset of the gods join'd on him to be brief and that the man is not so laid them to the duke of the fourth denial or not not not so much reasons
